<table align="center">
  <td align="center"><a target="_blank" href="https://colab.research.google.com/github/AU-DeepLearning-2020/Assignment6/blob/master/RL.ipynb">
        <img src="http://introtodeeplearning.com/images/colab/colab.png?v2.0"  style="padding-bottom:5px;" />Run in Google Colab</a></td>
</table>

# Copyright Information

**Parts of this lab are based on Kaggle kernels.**

# Lab 1 - Part1: Linear Regression

![Linear Regression]()

## 1.1.1 Problem Statement

The problem we are trying to solve here is finding a new house which is suitable to our needs and the budget we assigned. The client who wants to buy the new house did her research and found some houses. She wrote the details of each house she visited including location, sale condition, sale type, house price, among others. She needs some help to know how much she is expected to pay to get a house that conforms with her specific needs.

Your task is to build a linear regression model that helps her to predict the house price depending on the given attributes she collected.

## 1.1.2 Problem Details

Let's dive into the code, explain it and show you the parts you need to fill!

### 1.1.2.1 Import Needed packages

In [9]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib

import matplotlib.pyplot as plt
from scipy.stats import skew
from scipy.stats.stats import pearsonr
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.regularizers import l1, l2
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

### 1.1.2.2 Configure Notebook

In [10]:
%config InlineBackend.figure_format = 'retina' #set 'png' here when working on notebook
%matplotlib inline

### 1.1.2.3 Work on the Dataset

This dataset contains 80 features that demonstrate the state of the house and our target which is the house price.

We begin by loading the train and test splits of the dataset using pandas.

In [11]:
train = pd.read_csv("lab1_housing_train.csv")
test = pd.read_csv("lab1_housing_test.csv")

You can have a look at the train split of the dataset using the head command. I very much encourage you to have a deeper look on the dataset file.

In [ ]:
train.head()

Data preprocessing:
* First I'll transform the skewed numeric features by taking log(feature + 1) - this will make the features more normal
* Create Dummy variables for the categorical features
* Replace the numeric missing values (NaN's) with the mean of their respective columns



In [14]:
# Concatenate all the data
# We do this to be able to preprocess on the whole dataset
all_data = pd.concat((train.loc[:,'MSSubClass':'SaleCondition'],
                      test.loc[:,'MSSubClass':'SaleCondition']))

# Log transform the target y in training data - by reference inside all
train["SalePrice"] = np.log1p(train["SalePrice"])

# Log transform skewed numeric features:

# Get Numerical Fields
numeric_feats = all_data.dtypes[all_data.dtypes != "object"].index 

skewed_feats = train[numeric_feats].apply(lambda x: skew(x.dropna())) #compute skewnessc
skewed_feats = skewed_feats[skewed_feats > 0.75] # Get Skewed Columns
skewed_feats = skewed_feats.index # Get Skewed Columns indices

# Log scale skewed columns
# Normalize the skewed distribution for better regression
all_data[skewed_feats] = np.log1p(all_data[skewed_feats])

# Create Dummy variables for the categorical features 
all_data = pd.get_dummies(all_data) 

# Replace the numeric missing values (NaN's) with the mean of their respective columns
all_data = all_data.fillna(all_data.mean())

# Split the data to training & testing
X_train = all_data[:train.shape[0]]
X_test = all_data[train.shape[0]:]
y = train.SalePrice

# Standardize features by removing the mean and scaling to unit variance
# z = (x - u) / s
X_train = StandardScaler().fit_transform(X_train)

#split training data into training & validation, default splitting is 25% validation
X_tr, X_val, y_tr, y_val = train_test_split(X_train, y, random_state = 3)

### 1.1.2.4 Define your model here (TODO)

One important note you need to be aware of, linear regression is a neural network with only one perceptron (i.e. dense layer with one node) with a linear activation (i.e. no activation function). 

![One Perceptron Neural Network]()

Use this note to define a **sequential model of one dense layer with one unit using Tensorflow.Keras**.

In [15]:
# TODO: Define the Model using Tensorflow.Keras
model = Sequential([
    Dense(1, input_shape=(X_tr.shape[1],), activation='linear', kernel_regularizer=l2(0.05))
])

### 1.1.2.5 Compile your model and print a summary

In [ ]:
model.compile(loss = "mean_squared_error", optimizer = "Adam")
model.summary()

### 1.1.2.6 Train your model

Fit your model into the training data, use the validation data to be able to plot the loss decrement during the training. 

In [ ]:
hist = model.fit(X_tr, y_tr, validation_data = (X_val, y_val), epochs = 500)

And this is how you can predict an output for any number of inputs.

In [18]:
print(model.predict(X_test))

[[12.631396]
 [12.315755]
 [12.572445]
 ...
 [12.52041 ]
 [12.14757 ]
 [12.717263]]


### 1.1.2.7 Visualize Results

It's time to see how your model's progress during the training, If all is good, you will find the validation loss decreasing without neither overfitting nor underfitting.

In [ ]:
# Get training and test loss histories
training_loss = hist.history['loss']
val_loss = hist.history['val_loss']

# Create count of the number of epochs
epoch_count = range(1, len(training_loss) + 1)

# Visualize loss history
plt.figure()
plt.plot(epoch_count, training_loss, 'r--')
plt.plot(epoch_count, val_loss, 'b-')
plt.legend(['Training Loss', 'Val Loss'])
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()

## 1.1.3 Conclusion:

That's it! Congratulations on training a linear regression model. 

Make sure you finish the second part of the assignment and deliver all the requirements for the submission.

